# Deep NLP - Word Embeddings

Think back to NLP as we've understood it so far.

If we've had some luck with NLP modeling, likely with a NaiveBayes algorithm, we were able to illustrate some correlations between words and some other feature of interest.

But to whatever extent that our models were able to make connections and pick up on correlations, they did this *without any understanding of the **meaning** of the words in question*.

Let's think for a minute about words and objective meanings!

We can make sense of meaning for computational purposes by thinking about meaning in terms of similarity, i.e. thinking about meaning *holistically*.

Q. Is there any precedent for this way of thinking about meaning? <br/>
A. [Yes](https://plato.stanford.edu/entries/meaning-holism/#ArgForMeaHol)

So what will this look like for us?

*Remember cosine similarity?*

$\rightarrow$We'll have much the same idea here: Associate each word with values along particular dimensions in a multi-dimensional space. If we had a dimension for *softness*, for example, then pillows and marshmallows would score higher on it than rocks and bricks.

In [1]:
import gensim
import numpy as np

What is Gensim? See [here](https://en.wikipedia.org/wiki/Gensim) and [here](https://radimrehurek.com/gensim/). But, basically, gensim is a package with lots of topic-modeling and NLP tools, inlcuding Word2Vec.

Find the data [here!](https://drive.google.com/file/d/0BwT5wj_P7BKXb2hfM3d2RHU1ckE/view) (Just click 'Download')

In [129]:
# Reading in the data

import json

with open('JEOPARDY_QUESTIONS1.json') as f:
    data = json.load(f)

In [1]:
type(data)

In [2]:
len(data)

In [3]:
# Let's look at the first element in our list

data[0]

In [4]:
# How many words do we have?

len(data[0]['question'])

In [5]:
# Let's try that again!

data[0]['question'].split(' ')

In [6]:
len(data[0]['question'].split(' '))

In [7]:
length = 0
for clue in data:
    length += len(clue['question'].split(' '))
length

## Using Word2Vec

In [137]:
# Word2Vec requires that our text have the form of a list
# of 'sentences', where each sentence is itself a list of
# words. How can we put our _Jeopardy!_ clues in that shape?

import string
text = []

for clue in data:
    sentence = clue['question'].translate(str.maketrans('', '',
                                                        string.punctuation)).split(' ')
    
    new_sent = []
    for word in sentence:
        new_sent.append(word.lower())
    
    text.append(new_sent)

In [8]:
# Let's check the new structure of our first clue

text[0]

In [255]:
# Constructing the model is simply a matter of
# instantiating a Word2Vec object.

model = gensim.models.Word2Vec(text, sg=1)

See doc for note about seeding and reproducibility!

Continuous Bag of Words vs. Skipgram

<a href="https://www.researchgate.net/figure/Illustration-of-the-Skip-gram-and-Continuous-Bag-of-Word-CBOW-models_fig1_281812760"><img src="https://www.researchgate.net/profile/Wang_Ling/publication/281812760/figure/fig1/AS:613966665486361@1523392468791/Illustration-of-the-Skip-gram-and-Continuous-Bag-of-Word-CBOW-models.png" alt="Illustration of the Skip-gram and Continuous Bag-of-Word (CBOW) models."/></a>

[More on Skipgram](https://towardsdatascience.com/word2vec-skip-gram-model-part-1-intuition-78614e4d6e0b)

In [9]:
# To train, call 'train()'!

model.train(text, total_examples=model.corpus_count, epochs=model.epochs)

In [10]:
# Checking word count

model.corpus_total_words

## model.wv

In [11]:
# The '.wv' attribute stores the word vectors

model.wv

In [12]:
# The vectors are keyed by the words

model.wv['child']

### model.wv methods
#### 'most_similar()' and 'similarity()'

In [13]:
model.wv.most_similar('furniture')

In [14]:
model.wv.similarity('furniture', 'jewelry')

In [15]:
model.wv.most_similar(positive=['cat', 'animal', 'pet', 'mammal'])

Consider the following 'equations':

King + Woman - Man = x

Brother + Woman - Man = y

What values would you suggest for x and y here?

Clearly, getting good answers to these equations depends on understanding the *meanings* of the underlying words.

Or does it? The `most_similar()` method takes a 'negative' parameter as well as a 'positive' one, so we can consult our trained word vectors to see how they would answer these questions!

In [16]:
model.wv.most_similar(positive=['cat', 'animal'], negative='pet')

In [17]:
model.wv.most_similar(positive=['king', 'woman'], negative='man')

In [1]:
model.wv.most_similar(positive=['brother', 'woman'], negative='man')

In [18]:
model.wv.most_similar(positive='usa')

In [19]:
model.wv.most_similar('canada')

In [20]:
model.wv.most_similar('shakespeare')

In [21]:
model.wv.most_similar('greg')

In [22]:
model.wv.most_similar('jefferson')

In [23]:
model.wv.most_similar('washington')

In [24]:
model.wv.most_similar(positive=['president', 'germany'], negative='usa')

In [25]:
model.wv.most_similar(positive=['president', 'france'], negative='usa')

#### 'doesnt_match()'

In [26]:
model.wv.doesnt_match(['breakfast', 'lunch', 'frog', 'food'])

In [27]:
model.wv.doesnt_match(['lunch', 'this'])

In [28]:
model.wv.doesnt_match(['tree', 'flower', 'bush', 'plant', 'toothbrush'])

In [29]:
model.wv.doesnt_match(['tree', 'flower', 'plant', 'toothbrush'])

#### 'closer_than()'

In [30]:
# Which words are closer to 'king' than 'queen' is?

model.wv.closer_than('king', 'queen')

#### 'distance()'

In [ ]:
# For this it will make more sense to
# normalize our vectors.

model.init_sims(replace=True)

In [31]:
model.wv.distance('king', 'king')

In [32]:
model.wv.distance('joy', 'happiness')

#### 'evaluate_word_analogies()'

The `evaluate_word_analogies()` method takes in a string of quadruples, properly formatted (see [here](https://radimrehurek.com/gensim/models/keyedvectors.html)), and returns a list of dictionaries. Each dictionary has two keys: 'correct' and 'incorrect', the values for which are lists of the analogies that the model correctly or incorrectly predicted.

Check out [this text file](https://raw.githubusercontent.com/nicholas-leonard/word2vec/master/questions-words.txt)!

In [249]:
relatives = model.wv.evaluate_word_analogies("""https://raw.githubusercontent.com/nicholas-leonard/word2vec/master/questions-words.txt""")[1][4]

In [33]:
len(relatives['correct'])

In [34]:
len(relatives['incorrect'])

In [35]:
relatives['correct'][:5]

In [36]:
relatives['incorrect'][:5]

## Stanford Parser
https://nlp.stanford.edu/software/lex-parser.shtml <br/>
http://nlp.stanford.edu:8080/parser/